In [ ]:
import torch
import torch.nn as nn
import math

# 포지셔널 인코딩 클래스
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# 시계열 트랜스포머 모델 클래스
class TimeSeriesTransformerModel(nn.Module):
    def __init__(self, num_features, d_model, n_heads, num_encoder_layers, d_ff, dropout_rate):
        super(TimeSeriesTransformerModel, self).__init__()
        self.d_model = d_model
        self.linear = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout_rate)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model, n_heads, d_ff, dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        
        self.out = nn.Linear(d_model, 1)

    def forward(self, src):
        src = self.linear(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = torch.sigmoid(self.out(output[-1]))  # 마지막 시점의 출력만 사용
        return output

In [ ]:
# 모델 초기화
model = TimeSeriesTransformerModel(
    num_features=10,  # 예를 들어, 10개의 특징
    d_model=512,
    n_heads=8,
    num_encoder_layers=6,
    d_ff=2048,
    dropout_rate=0.1
)

# 손실 함수 및 최적화
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)